# PLANTILLA TRANSFORMACIÓN DE DATOS

**IMPORTANTE**: Recuerda hacer una copia de esta plantilla para no machacar la original.

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

2.- Nombrar los ficheros de datos.

In [2]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [3]:
cat = pd.read_pickle( '../../02_Datos/03_Trabajo/' + nombre_cat).reset_index(drop = True)
num = pd.read_pickle('../../02_Datos/03_Trabajo/' + nombre_num).reset_index(drop = True)

4.- Separar la target.

In [6]:
target = num[['compra']].copy().reset_index(drop=True)

In [7]:
num.visitas_total.value_counts()

visitas_total
2     1042
3      926
4      747
5      506
0      364
6      303
1      194
7      185
8      138
9       98
10      74
11      61
12      32
13      29
14      25
16      15
18      10
17       9
15       9
20       6
23       5
27       4
25       4
21       4
19       3
24       3
29       2
26       2
50       2
22       2
43       1
28       1
Name: count, dtype: Int64

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [8]:
var_ohe = ['origen','fuente','ult_actividad','ambito','ocupacion','descarga_lm']

#### Instanciar

In [9]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

#### Entrenar y aplicar

In [10]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [11]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

In [12]:
df=pd.concat([cat_ohe,num], axis=1)

## REESCALAR VARIABLES

### Con Min-Max

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4806 entries, 0 to 4805
Data columns (total 44 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   origen_API                                4806 non-null   float64
 1   origen_Landing Page Submission            4806 non-null   float64
 2   origen_Lead Add Form                      4806 non-null   float64
 3   origen_OTROS                              4806 non-null   float64
 4   fuente_Chat                               4806 non-null   float64
 5   fuente_Direct Traffic                     4806 non-null   float64
 6   fuente_Google                             4806 non-null   float64
 7   fuente_OTROS                              4806 non-null   float64
 8   fuente_Organic Search                     4806 non-null   float64
 9   fuente_Reference                          4806 non-null   float64
 10  ult_actividad_Chat Conversation     

#### Variables a reescalar con Min-Max

In [14]:
var_mms = df.iloc[:,39:].columns

#### Instanciar

In [15]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [16]:
df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [17]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

## UNIFICAR DATASETS REESCALADOS

### Crear una lista con los dataframes a incluir en el tablón analítico

In [18]:
incluir = [df.id,cat_ohe, df_mms,target]

### Unir todos los dataframes en el tablón analítico

In [19]:
df_tablon = pd.concat(incluir, axis = 1)

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [20]:
#Definir los nombres del archivo
ruta_df_tablon =  '../../02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [21]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)